# ntfy Survey Reminder - Live Demo (Google Colab)

This notebook is designed for Google Colab. It focuses on safe planning first
and only sends real notifications if you explicitly configure ntfy.


## Colab setup
Replace `<REPO-URL>` with your repository URL, then run this cell.


In [ ]:
!git clone <REPO-URL>
%cd ntfy-reminder


## Konfigurationsbearbeitung (Standardwerte)

Diese Werte entsprechen den CLI-Argumenten in `run.py` und den Standardwerten in `ntfy_reminder/config.py`. Wenn Sie sie nicht ändern, werden die Standardwerte verwendet.

- `start`, `end`  
    Definieren den Planungszeitraum als Start- und Enddatum (Format YYYY-MM-DD).

- `per_day`  
    Anzahl der Erinnerungen, die pro Tag geplant werden sollen.

- `min_gap`  
    Minimale Lücke zwischen zwei Erinnerungen in Minuten (verhindert zu kurze Abstände).

- `mode`  
    Planungsmodus: `"interval"` oder `"windows"`. `"interval"` verteilt Erinnerungen gleichmäßig innerhalb von `interval`; `"windows"` plant Erinnerungen innerhalb der definierten `windows`.

- `interval`  
    Tageszeitspanne im Format `HH:MM-HH:MM` (z. B. `08:00-20:00`), innerhalb der bei `mode="interval"` die Erinnerungen verteilt werden.

- `windows`  
    Komma-getrennte Zeitfenster (z. B. `08:00-10:00,10:00-12:00`), in denen bei `mode="windows"` Erinnerungen geplant werden.

- `participant_id`  
    Eindeutige Kennung für Teilnehmende; erzeugt deterministische Variationen (z. B. bei Seed/Randomisierung) und wird für Dateinamen verwendet.

Hinweis: systemd-Templates erwarten Dateien im Format `out/<id>_schedule.json`.


In [2]:
# Customize these values (or keep defaults)
participant_id = "student01"
start = "2026-01-21"
end = "2026-01-22"
per_day = 5
min_gap = 60
seed = 123  # set to None for random
mode = "interval"  # or "windows"
interval = "08:00-20:00"
windows = "08:00-10:00,10:00-12:00,12:00-14:00,14:00-16:00,16:00-18:00"
out_file = f"out/{participant_id}_schedule.json"


## Quick dry-run (safe)
This uses the values above to plan reminders without sending anything.
If you keep the defaults, it matches the original configuration.


In [3]:
import subprocess

cmd = [
    "python",
    "run.py",
    "--start",
    start,
    "--end",
    end,
    "--per-day",
    str(per_day),
    "--min-gap",
    str(min_gap),
    "--mode",
    mode,
    "--participant-id",
    participant_id,
    "--out",
    out_file,
]

if seed is not None:
    cmd += ["--seed", str(seed)]

if mode == "windows":
    cmd += ["--windows", windows]
else:
    cmd += ["--interval", interval]

cmd += [
    "plan",
    "--dry-run",
    "--explain",
]

subprocess.run(cmd, check=True)


[explain] participant_id='student01' -> pid_hash_int=13061395404846
[explain] final seed=13061272473119 (base_seed=123)
[explain] Seed gesetzt auf: 13061272473119
[explain] erlaubte Minuten pro Tag: 720 (aus 1 Fenster(n))
[explain] 2026-01-21: gewählt -> 08:11, 09:37, 12:23, 15:34, 18:11
[explain] 2026-01-22: gewählt -> 08:06, 12:31, 14:56, 17:30, 19:37
Geplante Zeitpunkte:
  ID 1: 2026-01-21 #1/5 @ 08:11 (2026-01-21T08:11)
  ID 2: 2026-01-21 #2/5 @ 09:37 (2026-01-21T09:37)
  ID 3: 2026-01-21 #3/5 @ 12:23 (2026-01-21T12:23)
  ID 4: 2026-01-21 #4/5 @ 15:34 (2026-01-21T15:34)
  ID 5: 2026-01-21 #5/5 @ 18:11 (2026-01-21T18:11)
  ID 6: 2026-01-22 #1/5 @ 08:06 (2026-01-22T08:06)
  ID 7: 2026-01-22 #2/5 @ 12:31 (2026-01-22T12:31)
  ID 8: 2026-01-22 #3/5 @ 14:56 (2026-01-22T14:56)
  ID 9: 2026-01-22 #4/5 @ 17:30 (2026-01-22T17:30)
  ID 10: 2026-01-22 #5/5 @ 19:37 (2026-01-22T19:37)

Gespeichert in: out/student01_schedule.json

[dry-run] Kein Versand.


CompletedProcess(args=['python', 'run.py', '--start', '2026-01-21', '--end', '2026-01-22', '--per-day', '5', '--min-gap', '60', '--mode', 'interval', '--participant-id', 'student01', '--out', 'out/student01_schedule.json', '--seed', '123', '--interval', '08:00-20:00', 'plan', '--dry-run', '--explain'], returncode=0)

## Optional: configure ntfy (.env)
To test real sending, create `config/ntfy.env`.
If you leave the defaults below unchanged, it matches `config/ntfy.env.example`.
Replace `NTFY_TOPIC`, `NTFY_TITLE`, and `NTFY_MESSAGE` to send your own test notification.


In [4]:
from pathlib import Path

env_path = Path("config/ntfy.env")
env_text = (
    "export NTFY_TOPIC=\"NTFY_TOPIC\"\n"
    "export NTFY_TITLE=\"Umfrage-Reminder {k}/{per_day}\"\n"
    "export NTFY_MESSAGE=\"Bitte nimm kurz an der Umfrage teil: {url}\"\n"
    "\n"
    "# Optional\n"
    "export SURVEY_URL_TEMPLATE=\"https://www.soscisurvey.de/dbd25-template//?r={id}\"\n"
    "# export NTFY_SERVER=\"https://ntfy.sh\"\n"
    "# export NTFY_MARKDOWN=\"1\"\n"
)
env_path.write_text(env_text)
print(f"Wrote {env_path}")


Wrote config/ntfy.env


## Optional: send a test notification
Only run this if `config/ntfy.env` has real values.


In [ ]:
!python run.py send 1 --explain


## Export schedule.json and .env
You can send these files back to the instructor.
The download helper only works in Colab.


In [5]:
from pathlib import Path

files_to_export = [Path(out_file), Path("config/ntfy.env")]

try:
    from google.colab import files
    for p in files_to_export:
        if p.exists():
            files.download(str(p))
        else:
            print(f"Missing: {p}")
except Exception:
    print("Not running in Colab.")


Not running in Colab.
